## Part1: Prepare Dataset

In [1]:
#Download packages
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from bs4.element import Comment
import re
import warnings
warnings.filterwarnings("ignore")
#Read dataset
data = pd.read_csv('Artworks-2.csv')
try:
    description_data = pd.read_excel('description_data (2).xlsx')
except:
    print('Description data not available')

In [ ]:
#data[['Title', 'Artist', 'Nationality','BeginDate', 'EndDate', 'Gender', 'Date', 'Medium',
#      'DateAcquired', 'URL']].isnull().sum()

In [2]:
data = data.replace(to_replace ='^Untitled', value = '', regex = True)
data['Gender'] = data['Gender'].str.replace(r"\(","")
data['Gender'] = data['Gender'].str.replace(r"\)","")
data['Nationality'] = data['Nationality'].str.replace(r"\(","")
data['Nationality'] = data['Nationality'].str.replace(r"\)","")
data['Title'] = data['Title'].astype(str)
data['Title'] = data['Title'].str.replace(r"\,","")
data['Medium'] = data['Medium'].str.replace(r"\,","")

In [3]:
data = data.dropna(subset=['URL', 'Title', 'Medium'])
#description_data.rename(columns= { 'article_number': 'Article_Number', 'article_url': 'Article_URL', 'article_body': 'Article_Body'}, inplace = True, errors='raise')
#description_data.to_excel('description_data.xlsx')

In [4]:
#print(data.size,"X", data.shape)

In [5]:
# -----------NOT USED------------
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head',
                               'title', 'meta', '[document]', 'div','span', 'blockquote']:
        return False
    if isinstance(element, Comment):
        return False
    return True


def text_from_article(body):
    soup = BeautifulSoup(body, 'html.parser')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts) # remove html tags  
    return u" ".join(t.strip() for t in visible_texts)

#text = text_from_article(page) 
#print(text)

In [6]:
# initializing tag 
tag = "p"

def first_pass(data):
    tag = "p"
    article_URL = []
    article_body = []
    article_number = []
    for i in range(0,100):
        url = data.URL[i]
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        try:
            page = urlopen(req).read()
            soup = BeautifulSoup(page, 'html.parser')
            description = soup.findAll(class_="main-content")
        except:
            description = ""
        reg_str = "<" + tag + ">(.*?)</" + tag + ">"
        res = re.findall(reg_str, str(description[0])) 
        article_body.append(str(res))
        article_URL.append(url)
        article_number.append(i)
    description_data = pd.DataFrame({ 'Article_Number' : article_number, 'Article_URL' : article_URL, 'Article_Body' : article_body})
    description_data.to_excel('description_data.xlsx')
    return description_data

def scrap_description(data):
    try:
        description_data = pd.read_excel('description_data.xlsx')
    except:
        description_data = first_pass(data)
    n = description_data['Article_Number'].tail(1).values[0]
    tag = "p"
    article_URL = description_data['Article_URL'].to_list()
    article_body = description_data['Article_Body'].to_list()
    article_number = description_data['Article_Number'].to_list()
    for i in range(n, n + 1000):
        try:
            url = data.URL[i]
            if url not in article_URL:
                req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                try:
                    page = urlopen(req).read()
                    soup = BeautifulSoup(page, 'html.parser')
                    description = soup.findAll(class_="main-content")
                except:
                    description = [""]
                reg_str = "<" + tag + ">(.*?)</" + tag + ">"
                res = re.findall(reg_str, str(description[0]))
                article_body.append(str(res))
                article_URL.append(url)
                article_number.append(i)
        except:
            continue
    description_data = pd.DataFrame({ 'Article_Number' : article_number, 'Article_URL' : article_URL, 'Article_Body' : article_body})
    description_data.to_excel('description_data.xlsx')
    return description_data        

#### Scraping description information and adding it to original dataset (In the original dataset, the description column only shows the url)

In [7]:
%%time
#n = 0 
#while n < 10 : 
#    description_data = scrap_description(data)
#    n += 1

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 8.11 µs


In [8]:
description_data.shape

(59484, 4)

In [9]:
good_index = []
for i in range(0,len(description_data)):
    if description_data.loc[i]['Article_Body'].startswith("['If you would like ") | description_data.loc[i]['Article_Body'].startswith("[\'In 2018–19, MoMA collaborated") is False:
        good_index.append(i)
description_data = description_data.iloc[good_index, ]

In [10]:
description_data.shape

(11261, 4)

In [11]:
%%time
full_data = data.merge(description_data, left_on = 'URL', right_on = 'Article_URL', how = 'inner')
drop_col = ['ConstituentID', 'BeginDate', 'EndDate', 'Date', 'Dimensions', 'CreditLine', 'ArtistBio', 'AccessionNumber', 'DateAcquired', 'Article_Number', 'Duration (sec.)', 'Seat Height (cm)',
           'Width (cm)', 'Weight (kg)', 'Length (cm)', 'Height (cm)', 'Diameter (cm)', 'Depth (cm)', 'Circumference (cm)', 'ThumbnailURL', 'URL', 'ObjectID', 'Cataloged']
full_data.drop(columns= drop_col, inplace = True)
full_data = full_data.sample(frac=0.8, random_state=257)
full_data.to_csv('AggregatedData.csv')

CPU times: user 265 ms, sys: 41.3 ms, total: 306 ms
Wall time: 323 ms


In [12]:
print(full_data.shape)

(9009, 10)


## Part2: Apply Sentiment Analysis

In [14]:
! pip install wordcloud

     |████████████████████████████████| 220 kB 762 kB/s eta 0:00:01
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp38-cp38-macosx_10_9_x86_64.whl size=160402 sha256=194d52df7420ad0ad869e021f3a09c746f7df21a9ce47b836ac5ea9a4540625e
  Stored in directory: /Users/judy/Library/Caches/pip/wheels/4d/3f/0d/a2ba9b7895c9f1be89018b3141c3df3d4f9c786c882ccfbc3b
Successfully built wordcloud


In [19]:
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [16]:
!pip install textblob

     |████████████████████████████████| 636 kB 4.9 MB/s eta 0:00:01


In [18]:
!pip install vaderSentiment

     |████████████████████████████████| 125 kB 4.5 MB/s eta 0:00:01


In [35]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
#replace numbers with # to allow lemmatization of 'Title'
def replaceDigits(text):
    return_text = re.sub(r'[0-9]','#',text)
    return return_text

def get_sentiment(data, input_column, output_column):
    stop = stopwords.words('english')
    st = PorterStemmer()
    analyzer = SentimentIntensityAnalyzer()
    data[output_column] = data[input_column].apply(replaceDigits)
    data[output_column] = data[output_column].apply(lambda x: ' '.join([x for x in x.split() if x not in (stop)]))
    data[output_column] = data[output_column].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    desc_blob = [TextBlob(t) for t in data[output_column]]
    #add the sentiment metrics to the dataframe
    data[format(input_column + '_'+'Sentiment_Polarity')] = [b.sentiment.polarity for b in desc_blob]
    data[format(input_column + '_'+'Sentiment_Subjectivity')] = [b.sentiment.subjectivity for b in desc_blob]
    analyzer = SentimentIntensityAnalyzer()
    #Add VADER metrics to dataframe
    #data['compound'] = [analyzer.polarity_scores(v)['compound'] for v in data['processed_title']]
    data[format(input_column+'_'+'Neg')] = [analyzer.polarity_scores(v)['neg'] for v in data[output_column]]
    data[format(input_column+'_'+'Neu')] = [analyzer.polarity_scores(v)['neu'] for v in data[output_column]]
    data[format(input_column+'_'+'Pos')] = [analyzer.polarity_scores(v)['pos'] for v in data[output_column]]
    return data

In [21]:
%%time
new_data = get_sentiment(full_data, 'Title', 'Processed_Title')

CPU times: user 4.16 s, sys: 78.7 ms, total: 4.24 s
Wall time: 4.28 s


In [22]:
%%time
senti_data = get_sentiment(new_data, 'Article_Body', 'Processed_Body')

CPU times: user 27.9 s, sys: 100 ms, total: 28 s
Wall time: 28.2 s


In [23]:
senti_data.describe()

,Unnamed: 0,Title_Sentiment_Polarity,Title_Sentiment_Subjectivity,Title_Neg,Title_Neu,Title_Pos,Article_Body_Sentiment_Polarity,Article_Body_Sentiment_Subjectivity,Article_Body_Neg,Article_Body_Neu,Article_Body_Pos
count,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000,9009.000000
mean,37030.832279,0.015130,0.068352,0.043736,0.888297,0.034114,0.017691,0.131666,0.017809,0.959006,0.023184
std,18966.147134,0.128346,0.183599,0.139798,0.250287,0.130304,0.088668,0.201334,0.042412,0.071864,0.044479
min,11.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-0.800000,0.000000,0.000000,0.528000,0.000000
25%,17966.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.933000,0.000000
50%,50181.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,53012.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.286553,0.000000,1.000000,0.033000
max,59474.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.711111,1.000000,0.410000,1.000000,0.331000


In [24]:
senti_data.query('''`Title_Sentiment_Polarity` == -1''')

,Title,Artist,Nationality,Gender,Medium,Classification,Department,Unnamed: 0,Article_URL,Article_Body,...,Title_Sentiment_Subjectivity,Title_Neg,Title_Neu,Title_Pos,Processed_Body,Article_Body_Sentiment_Polarity,Article_Body_Sentiment_Subjectivity,Article_Body_Neg,Article_Body_Neu,Article_Body_Pos
4096,Don’t Be Cruel,Residents,American,,Video,Video,Film,29882,http://www.moma.org/collection/works/215748,[],...,1.0,0.655,0.345,0.000,[],0.000000,0.000000,0.00,1.000,0.000
6770,The Terrible Archer (L'archer terrible),James Ensor,Belgian,Male,Etching,Print,Prints & Illustrated Books,51361,http://www.moma.org/collection/works/62646,[],...,1.0,0.437,0.563,0.000,[],0.000000,0.000000,0.00,1.000,0.000
7076,The Flowers of Evil (Les Fleurs du mal),Odilon Redon,French,Male,Portfolio of nine photogravures (including cover),Print,Prints & Illustrated Books,51667,http://www.moma.org/collection/works/63101,[],...,1.0,0.423,0.577,0.000,[],0.000000,0.000000,0.00,1.000,0.000
5430,The Spirit of Evil is Speaking (Mahna no varua...,Paul Gauguin,French,Male,Woodcut,Print,Prints & Illustrated Books,50021,http://www.moma.org/collection/works/60959,[],...,1.0,0.396,0.450,0.153,[],0.000000,0.000000,0.00,1.000,0.000
523,But deliver us from evil (Sondern erlöse uns v...,Max Pechstein,German,Male,Woodcut from a portfolio of twelve woodcuts an...,Print,Prints & Illustrated Books,2821,http://www.moma.org/collection/works/109165,"[""Promising forgiveness and deliverance from e...",...,1.0,0.289,0.711,0.000,"[""promis forgiv deliver evil, lord' prayer res...",-0.075758,0.346212,0.12,0.817,0.063
4787,Canto XXVII: Circle Eight Bolgia 8 The Evil Co...,Robert Rauschenberg,American,Male,Solvent transfer drawing watercolor gouache an...,Drawing,Drawings,36708,http://www.moma.org/collection/works/36738,['Rauschenberg made one drawing for each Canto...,...,1.0,0.253,0.747,0.000,"['rauschenberg made one draw canto, section, d...",0.176667,0.333333,0.00,0.939,0.061
7278,Evil Glory (Mauvaise gloire),Odilon Redon,French,Male,Etching and drypoint,Print,Prints & Illustrated Books,51869,http://www.moma.org/collection/works/63325,[],...,1.0,0.595,0.405,0.000,[],0.000000,0.000000,0.00,1.000,0.000


In [25]:
senti_data.query('''`Article_Body_Sentiment_Polarity` <= 0''')

,Title,Artist,Nationality,Gender,Medium,Classification,Department,Unnamed: 0,Article_URL,Article_Body,...,Title_Sentiment_Subjectivity,Title_Neg,Title_Neu,Title_Pos,Processed_Body,Article_Body_Sentiment_Polarity,Article_Body_Sentiment_Subjectivity,Article_Body_Neg,Article_Body_Neu,Article_Body_Pos
5733,Hydra (L'Hydre),Germaine Richier,French,Female,Aquatint,Print,Prints & Illustrated Books,50324,http://www.moma.org/collection/works/61354,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
8638,Normandy Landing,Louise Bourgeois,American,Female,Line block,Print,Prints & Illustrated Books,53229,http://www.moma.org/collection/works/65033,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
10497,Typenmöbel (Standardized furniture),Ernst Mumenthaler,Swiss,Male,Lithograph,Design,Architecture & Design,55088,http://www.moma.org/collection/works/6754,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
8591,Good Time Charley,Jasper Johns,American,Male,Lithograph,Print,Prints & Illustrated Books,53182,http://www.moma.org/collection/works/64978,[],...,0.6,0.0,0.408,0.592,[],0.0,0.0,0.0,1.0,0.0
8000,Biale Malzenstwo,Franciszek Starowieyski,Polish,Male,Offset lithograph,Design,Architecture & Design,52591,http://www.moma.org/collection/works/6421,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6597,Henriette,Louise Bourgeois,American,Female,Lithograph and digital print with hand additions,Print,Prints & Illustrated Books,51188,http://www.moma.org/collection/works/62450,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
6374,Self-Portrait (Selbstbildnis),Max Liebermann,German,Male,Lithograph,Print,Prints & Illustrated Books,50965,http://www.moma.org/collection/works/62149,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
4118,Residents on Night Music,Residents,American,,Video,Video,Film,29905,http://www.moma.org/collection/works/215771,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0
1890,Zócalo de la tragedia,Beatriz González,Colombian,Female,One from a set of six linoleum cuts,Print,Prints & Illustrated Books,13157,http://www.moma.org/collection/works/145859,[],...,0.0,0.0,1.000,0.000,[],0.0,0.0,0.0,1.0,0.0


In [26]:
full_data['Title_Sentiment_Polarity'] = senti_data['Title_Sentiment_Polarity']
full_data['Article_Body_Sentiment_Polarity'] = senti_data['Article_Body_Sentiment_Polarity']

## Part 3: Conduct Emotion Extraction with NRC

In [27]:
!pip install nrclex
from nrclex import NRCLex

     |████████████████████████████████| 396 kB 2.5 MB/s eta 0:00:01
  Created wheel for nrclex: filename=NRCLex-3.0.0-py3-none-any.whl size=43310 sha256=5a3c913bed8755f4030e928b16afca76ca2499b630e8b7e44e30729de82ea633
  Stored in directory: /Users/judy/Library/Caches/pip/wheels/83/95/c0/42b43fb15eb48e4f5a67cba8915540cb2783591c59c037a9e5
Successfully built nrclex


In [28]:
def get_emotion(data, input_column, emotion_column):
    stop = stopwords.words('english')
    st = PorterStemmer()
    analyzer = SentimentIntensityAnalyzer()
    output_column = format('Processed'+'_'+input_column)
    data[output_column] = data[input_column].apply(replaceDigits)
    data[output_column] = data[output_column].apply(lambda x: ' '.join([x for x in x.split() if x not in (stop)]))
    data[output_column] = data[output_column].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    #Using NRCLex
    emotion = []
    for text in data[output_column]:
        e_object = NRCLex(text)
        if len(e_object.top_emotions) < 10:
            emotion.append(e_object.top_emotions[0][0])
        else:
            emotion.append('neutral')
    data[emotion_column] = emotion
    return data

In [29]:
%%time
nrc_data = get_emotion(full_data, 'Article_Body', 'Description_Emotion')

CPU times: user 19.5 s, sys: 79 ms, total: 19.6 s
Wall time: 19.7 s


In [30]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/judy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [31]:
%%time
full_data = get_emotion(nrc_data, 'Title', 'Title_Emotion')

CPU times: user 3.88 s, sys: 19.3 ms, total: 3.9 s
Wall time: 3.92 s


In [32]:
full_data['Description_Emotion'].value_counts()

neutral         5809
positive        2056
trust            446
negative         366
fear             123
anticipation     100
anger             39
surprise          38
joy               19
sadness           13
Name: Description_Emotion, dtype: int64

In [33]:
full_data['Title_Emotion'].value_counts()

neutral         5601
fear            1076
positive         769
trust            648
negative         379
surprise         169
anticipation     154
anger            123
sadness           68
joy               16
disgust            6
Name: Title_Emotion, dtype: int64

In [34]:
pd.crosstab(index = full_data['Title_Emotion'], columns = full_data['Description_Emotion'])

Description_Emotion,anger,anticipation,fear,joy,negative,neutral,positive,sadness,surprise,trust
Title_Emotion,,,,,,,,,,
anger,2,2,1,0,1,100,16,0,0,1
anticipation,0,7,9,0,3,100,30,0,0,5
disgust,0,0,0,0,0,6,0,0,0,0
fear,1,6,37,2,81,661,278,2,0,8
joy,1,0,0,1,0,14,0,0,0,0
negative,0,4,12,0,51,221,83,0,0,8
neutral,33,67,55,6,151,3715,1194,5,36,339
positive,1,6,6,6,44,413,265,0,0,28
sadness,0,2,1,0,2,43,13,6,0,1


## Part4: Apply Topic Modeling on "Title" Column

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np

stopwords = set(nltk.corpus.stopwords.words('english'))

In [36]:
def get_topics_titles(data):
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                max_features=100,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 5000, 
                                min_df = 10,
                                ngram_range=(2,4)
                               )
    dtm_tf_body = tf_vectorizer.fit_transform(data['Title'])
    lda_tf_body = LatentDirichletAllocation(n_components=3,
                                   max_iter=50,
                                   learning_method='online', 
                                   random_state = 0)
    lda_output = lda_tf_body.fit_transform(dtm_tf_body)
    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_tf_body.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.around(lda_output,2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    data['Title_Topic'] = dominant_topic
    return data

def get_topics_description(data):
    tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                max_features=100,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 500, 
                                min_df = 10,
                                ngram_range=(2,4)
                               )
    dtm_tf_body = tf_vectorizer.fit_transform(data['Article_Body'])
    lda_tf_body = LatentDirichletAllocation(n_components=3,
                                   max_iter=50,
                                   learning_method='online', 
                                   random_state = 0)
    lda_output = lda_tf_body.fit_transform(dtm_tf_body)
    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_tf_body.n_components)]
    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]
    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.around(lda_output,2), columns=topicnames, index=docnames)
    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    data['Description_Topic'] = dominant_topic
    return data

In [37]:
%%time
full_data = get_topics_titles(full_data)

CPU times: user 30.7 s, sys: 184 ms, total: 30.9 s
Wall time: 31.4 s


In [38]:
%%time
full_data = get_topics_description(full_data)

CPU times: user 34.5 s, sys: 241 ms, total: 34.7 s
Wall time: 35.3 s


## Part5: Similarity Ranking on "Title" Column

In [39]:
import gensim, operator
from scipy import spatial
from gensim.models import KeyedVectors

In [41]:
model_path = '/Users/judy/Desktop/cu/NLP/'

def load_wordvec_model(modelName, modelFile, flagBin):
    print('Loading ' + modelName + ' model...')
    model = KeyedVectors.load_word2vec_format(model_path + modelFile, binary=flagBin)
    print('Finished loading ' + modelName + ' model...')
    return model

model_word2vec = load_wordvec_model('Word2Vec', 'GoogleNews-vectors-negative300.bin', True)

Loading Word2Vec model...
Finished loading Word2Vec model...


In [42]:
def vec_similarity(input1, input2, vectors):
    term_vectors = [np.zeros(300), np.zeros(300)]
    terms = [input1, input2]
        
    for index, term in enumerate(terms):
        for i, t in enumerate(term.split(' ')):
            try:
                term_vectors[index] += vectors[t]
            except:
                term_vectors[index] += 0
        
    result = (1 - spatial.distance.cosine(term_vectors[0], term_vectors[1]))
    if result is 'nan':
        result = 0
        
    return result

def vocab_check(vectors, words):
    
    output = list()
    for word in words:
        if word in vectors.vocab:
            output.append(word.strip())
            
    return output

# function calculates similarity between two strings using a particular word vector model
def calc_similarity(input1, input2, vectors):
    s1words = set(vocab_check(vectors, input1.split()))
    s2words = set(vocab_check(vectors, input2.split()))
    
    #add for error: At least one of the passed list is empty.
    try:
        output = vectors.n_similarity(s1words, s2words)
    except ZeroDivisionError:
        output=0
    return output

In [43]:
def rank_titles(data, search_title, vectors):
    og_titles = data['Title'].to_list()
    all_titles = data['Processed_Title'].to_list()
    all_body = data['Processed_Body'].to_list()
    all_URL = data['Article_URL'].to_list()
    search_title = search_title
    title_index = og_titles.index(search_title)
    try:
        search_description = data['Article_Body'][title_index]
    except:
        search_description = " "
    model = vectors
    new_titles = []
    scores_t = []
    scores_b = []
    URL = []
    del all_titles[title_index]
    del all_body[title_index]
    del all_URL[title_index]
    for i in range(len(all_titles)):
        if len(all_titles[i]) < 1 | len(all_body[i]) < 1:
            continue
        else:
            score_t = calc_similarity(search_title, all_titles[i], model)
            scores_t.append(score_t)
            score_b = calc_similarity(search_description, all_body[i], model)
            scores_b.append(score_b) 
            URL.append(all_URL[i])
    resultdf = pd.DataFrame(data = {'Score_Title' : scores_t, 'Score_Description': scores_b, 'Article_URL': URL})
    resultdf = data.merge(resultdf, on = 'Article_URL', how = 'inner')
    return resultdf

In [71]:
# 2965, 525, 5050, 5250, 1559, 2559
search_title = full_data['Title'][7881]
search_data = full_data[full_data['Title'] == search_title].head(1)
search_data[['Title', 'Article_Body', 'Department', 'Medium', 'Title_Topic', 'Title_Emotion', 'Description_Topic', 'Description_Emotion', 'Article_URL']]

,Title,Article_Body,Department,Medium,Title_Topic,Title_Emotion,Description_Topic,Description_Emotion,Article_URL
7881,Back of a Nude Woman (Femme nue de dos),[],Prints & Illustrated Books,Etching,0,neutral,0,neutral,http://www.moma.org/collection/works/64068


In [70]:
full_data[full_data["Artist"]=="Pablo Picasso"]

,Title,Artist,Nationality,Gender,Medium,Classification,Department,Unnamed: 0,Article_URL,Article_Body,...,Article_Body_Sentiment_Polarity,Article_Body_Sentiment_Subjectivity,Article_Body_Neg,Article_Body_Neu,Article_Body_Pos,Processed_Article_Body,Description_Emotion,Title_Emotion,Title_Topic,Description_Topic
10594,Armchair Woman No. 1 (La Femme au fauteuil No. 1),Pablo Picasso,Spanish,Male,Lithograph,Print,Prints & Illustrated Books,55185,http://www.moma.org/collection/works/67694,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,neutral,0,0
8301,The Maid in the Sculpture Studio (La Bonne dan...,Pablo Picasso,Spanish,Male,Etching,Print,Prints & Illustrated Books,52892,http://www.moma.org/collection/works/64631,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,positive,2,0
7881,Back of a Nude Woman (Femme nue de dos),Pablo Picasso,Spanish,Male,Etching,Print,Prints & Illustrated Books,52472,http://www.moma.org/collection/works/64068,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,neutral,0,0
10904,Bust of Young Woman in Three-Quarter View (Bus...,Pablo Picasso,Spanish,Male,Woodcut,Print,Prints & Illustrated Books,55608,http://www.moma.org/collection/works/68244,"[""This is a portrait of Fernande Olivier, Pica...",...,0.130803,0.395591,0.054,0.852,0.095,"[""thi portrait fernand olivier, picasso' lover...",positive,surprise,0,0
8382,The Minotaur's Repose: Champagne and Mistress ...,Pablo Picasso,Spanish,Male,Etching,Print,Prints & Illustrated Books,52973,http://www.moma.org/collection/works/64718,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,anger,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,Portrait of Françoise of the Long Neck I (Port...,Pablo Picasso,Spanish,Male,Aquatint and engraving printed in relief,Print,Prints & Illustrated Books,3367,http://www.moma.org/collection/works/110512,"[""In 1943 Picasso met Françoise Gilot, a young...",...,-0.083333,0.236667,0.059,0.890,0.051,"[""in #### picasso met françois gilot, young as...",positive,anticipation,0,0
2265,The Weeping Woman I (La Femme qui pleure I) st...,Pablo Picasso,Spanish,Male,Drypoint aquatint etching and scraper,Print,Prints & Illustrated Books,15632,http://www.moma.org/collection/works/154917,"['The <em>Weeping Woman, I</em> is an elaborat...",...,0.122917,0.444792,0.182,0.750,0.068,"['the <em>weep woman, i</em> elabor one centra...",negative,negative,0,0
7040,The Bull (Le Taureau) state XI,Pablo Picasso,Spanish,Male,Lithograph,Print,Prints & Illustrated Books,51631,http://www.moma.org/collection/works/63062,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,neutral,0,0
9504,Exposition 1952 Vallauris,Pablo Picasso,Spanish,Male,Woodcut,Print,Prints & Illustrated Books,54095,http://www.moma.org/collection/works/66153,[],...,0.000000,0.000000,0.000,1.000,0.000,[],neutral,neutral,0,0


In [72]:
ranked_data = rank_titles(data = full_data, search_title = search_title, vectors = model_word2vec)

In [73]:
ranked_data[['Title', 'Article_Body', 'Title_Topic', 'Title_Emotion', 'Description_Topic', 'Description_Emotion', 'Score_Title', 'Article_URL']].sort_values(by = 'Score_Title', ascending = False).head(15)

,Title,Article_Body,Title_Topic,Title_Emotion,Description_Topic,Description_Emotion,Score_Title,Article_URL
6871,Two Studies of a Nude (Deux études de nu),[],0,neutral,0,neutral,0.658457,http://www.moma.org/collection/works/60998
427,Study of a Nude (Etude de nu),[],0,neutral,0,neutral,0.652835,http://www.moma.org/collection/works/61826
7778,Firmin Gimiér and Un Satyre program for Mariag...,['Lautrec loved the theater and designed progr...,0,neutral,1,positive,0.646080,http://www.moma.org/collection/works/152676
2235,Study of an Upside-Down Nude (Étude de nu renv...,[],0,neutral,0,neutral,0.644077,http://www.moma.org/collection/works/61986
7123,Nude Woman Holding a Scarf (Femme Nue de dos t...,[],0,positive,0,neutral,0.631863,http://www.moma.org/collection/works/63642
3645,Nude Woman with Crossed Legs Crowned with Flow...,[],2,fear,0,neutral,0.618005,http://www.moma.org/collection/works/61396
100,Bust of Young Woman in Three-Quarter View (Bus...,"[""This is a portrait of Fernande Olivier, Pica...",0,surprise,0,positive,0.611853,http://www.moma.org/collection/works/68244
8232,Study of a Woman (Étude de femme),[],0,neutral,0,neutral,0.608712,http://www.moma.org/collection/works/62871
8716,At the Moulin de la Galette (Au Moulin de la G...,[],0,positive,0,neutral,0.605145,http://www.moma.org/collection/works/65375
5850,Motherhood (Maternité) from Essai dur l'histoi...,[],0,trust,0,neutral,0.603810,http://www.moma.org/collection/works/63645


In [74]:
ranked_data[['Title', 'Article_Body', 'Title_Topic', 'Title_Emotion', 'Description_Topic', 'Description_Emotion', 'Score_Description', 'Article_URL']].sort_values(by = 'Score_Description', ascending = False).head(15) #.iloc[:, 0]

,Title,Article_Body,Title_Topic,Title_Emotion,Description_Topic,Description_Emotion,Score_Description,Article_URL
0,Hydra (L'Hydre),[],0,fear,0,neutral,0,http://www.moma.org/collection/works/61354
6008,Cash from Chaos/Unicorns and Rainbows,[],0,fear,0,neutral,0,http://www.moma.org/collection/works/194852
6002,Nude with Blue Cushion (Nu au coussin bleu),[],0,positive,0,neutral,0,http://www.moma.org/collection/works/65265
6003,Spatial City project Perspective,"[""The Spatial City (<em>Ville spatiale</em>) i...",0,neutral,0,positive,0,http://www.moma.org/collection/works/104695
6004,Israel & Vietnam The Horrors of Wr A Dedicatio...,[],0,fear,0,neutral,0,http://www.moma.org/collection/works/66254
6005,Plaza de San Jose,[],0,neutral,0,neutral,0,http://www.moma.org/collection/works/66042
6006,Kilta Tableware,['The Kilta Tableware service is one of the mo...,0,neutral,0,positive,0,http://www.moma.org/collection/works/128887
6007,Foreign Parts,[],0,negative,0,neutral,0,http://www.moma.org/collection/works/162231
6009,Surprise Attack,['Primarily known for his large-scale lead and...,0,fear,0,negative,0,http://www.moma.org/collection/works/107734
6000,From the series 16 Variations of 2 Horizontal ...,['Sandback developed his signature line sculpt...,0,neutral,0,positive,0,http://www.moma.org/collection/works/118661


## Part6: Final Output 

In [75]:
#calculate ranking score
def rank_score(ranked_data, full_data, search_title):
    search_data = full_data[full_data['Title'] == search_title].head(1)
    scores = []
    for i in range(len(ranked_data)):
        score_t = 0
        score_d = 0
        if ranked_data['Title_Emotion'][i] == search_data['Title_Emotion'].values:
            score_t += 1
        if ranked_data['Description_Emotion'][i] == search_data['Description_Emotion'].values:
            score_d += 0.5
        if ranked_data['Title_Topic'][i] == search_data['Title_Topic'].values:
            score_t += 1
        if ranked_data['Description_Topic'][i] == search_data['Description_Topic'].values:
            score_d += 1
        score_t = (score_t+1) * ranked_data['Score_Title'][i]
        score_d = (score_d+1) * ranked_data['Score_Description'][i]
        final_score = score_t + score_d
        scores.append(final_score)
    ranked_data['Final_Score'] = scores
    ranked_data = ranked_data.sort_values(by = 'Final_Score', ascending = False)
    return ranked_data

In [76]:
final_ranked_data = rank_score(ranked_data = ranked_data, full_data = full_data, search_title = search_title)
final_ranked_data[['Title', 'Artist', 'Department', 'Medium', 'Title_Topic', 'Title_Emotion', 'Description_Topic', 'Description_Emotion', 'Article_URL']].head(15)

,Title,Artist,Department,Medium,Title_Topic,Title_Emotion,Description_Topic,Description_Emotion,Article_URL
6871,Two Studies of a Nude (Deux études de nu),Henri Matisse,Prints & Illustrated Books,Drypoint,0,neutral,0,neutral,http://www.moma.org/collection/works/60998
427,Study of a Nude (Etude de nu),Georges Braque,Prints & Illustrated Books,Etching,0,neutral,0,neutral,http://www.moma.org/collection/works/61826
7778,Firmin Gimiér and Un Satyre program for Mariag...,"Henri de Toulouse-Lautrec, Louis Anquetin",Prints & Illustrated Books,One from an album of 50 lithographs,0,neutral,1,positive,http://www.moma.org/collection/works/152676
2235,Study of an Upside-Down Nude (Étude de nu renv...,Henri Matisse,Prints & Illustrated Books,Etching with chine collé,0,neutral,0,neutral,http://www.moma.org/collection/works/61986
8232,Study of a Woman (Étude de femme),Henri de Toulouse-Lautrec,Prints & Illustrated Books,Lithograph with stencil additions,0,neutral,0,neutral,http://www.moma.org/collection/works/62871
8791,Theatre de la Renaissance Sarah Bernhardt La S...,Alphonse Mucha,Architecture & Design,Lithograph,0,neutral,0,neutral,http://www.moma.org/collection/works/6306
6640,La Terrasse (The Terrace) from Essai dur l'his...,Maurice Denis,Prints & Illustrated Books,Lithograph from a portfolio of 16 lithographs,0,neutral,0,neutral,http://www.moma.org/collection/works/63640
4802,Nude with Necklace (Nu au collier),Henri Matisse,Prints & Illustrated Books,Etching with chine collé,0,neutral,0,neutral,http://www.moma.org/collection/works/61176
6121,Nude Lying on Her Side Arms Under Head (Nu cou...,Henri Matisse,Prints & Illustrated Books,Etching with chine collé,0,neutral,0,neutral,http://www.moma.org/collection/works/62054
51,Study of a Head (Étude de tête) from Essai dur...,Démétrius Emmanuel Galanis,Prints & Illustrated Books,Lithograph from a portfolio of 16 lithographs,0,neutral,0,neutral,http://www.moma.org/collection/works/63653


In [77]:
ranked_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9008 entries, 0 to 9007
Data columns (total 30 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Title                                9008 non-null   object 
 1   Artist                               9008 non-null   object 
 2   Nationality                          9008 non-null   object 
 3   Gender                               9008 non-null   object 
 4   Medium                               9008 non-null   object 
 5   Classification                       9008 non-null   object 
 6   Department                           9008 non-null   object 
 7   Unnamed: 0                           9008 non-null   int64  
 8   Article_URL                          9008 non-null   object 
 9   Article_Body                         9008 non-null   object 
 10  Processed_Title                      9008 non-null   object 
 11  Title_Sentiment_Polarity      